In [38]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [39]:
dataset_dir = "dogs_cats"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

os.listdir(dataset_dir)

['test', 'train']

In [41]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# Only rescale pixel values for validation data
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [101]:
batch_size = 32

In [113]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode="binary"
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode="binary"
)

Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [120]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [121]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [123]:
model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    shuffle=True
)

Epoch 1/30
18/18 [==============================] - 15s 844ms/step - loss: 0.6929 - accuracy: 0.4883 - val_loss: 0.6928 - val_accuracy: 0.5500
Epoch 2/30
18/18 [==============================] - 12s 673ms/step - loss: 0.6939 - accuracy: 0.5171 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 3/30
18/18 [==============================] - 12s 655ms/step - loss: 0.6912 - accuracy: 0.5027 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 4/30
18/18 [==============================] - 11s 635ms/step - loss: 0.6885 - accuracy: 0.5332 - val_loss: 0.6870 - val_accuracy: 0.5286
Epoch 5/30
18/18 [==============================] - 11s 589ms/step - loss: 0.6758 - accuracy: 0.6014 - val_loss: 0.6864 - val_accuracy: 0.5286
Epoch 6/30
18/18 [==============================] - 11s 585ms/step - loss: 0.6617 - accuracy: 0.5889 - val_loss: 0.6736 - val_accuracy: 0.5929
Epoch 7/30
18/18 [==============================] - 11s 590ms/step - loss: 0.6393 - accuracy: 0.6409 - val_loss: 0.6669 - val_accuracy: 0.6000

In [124]:
# Evaluate model

test_loss, test_acc = model.evaluate(validation_generator, verbose=2)

print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

5/5 - 1s - loss: 0.7964 - accuracy: 0.6929 - 929ms/epoch - 186ms/step
Test accuracy: 0.6928571462631226
Test loss: 0.7963787913322449


In [132]:
# Dog.jpg

img = keras.preprocessing.image.load_img(
    "dog.jpg",
    target_size=(150, 150)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)


predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

1/1 [==============================] - 0s 24ms/step


In [133]:
# Save the model

model.save("dogs_cats.h5")

c:\Users\joshi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [137]:
# Load and test the model

model = keras.models.load_model("dogs_cats.h5")

img = keras.preprocessing.image.load_img(
    "dog.jpg",
    target_size=(150, 150)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)


predictions = model.predict(img_array)
print("This image most likely belongs to {}"
    .format("dog" if predictions[0] == 1 else "cat",)
)


1/1 [==============================] - 0s 64ms/step
This image most likely belongs to dog
